In [103]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression

In [219]:
X = load_wine(return_X_y=False, as_frame=True)

In [220]:
X['data'].rename(columns={'od280/od315_of_diluted_wines':'od280'}, inplace=True)

d:\Anaconda\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [221]:
rf = RandomForestClassifier(random_state=42, n_estimators=10)
rf.fit(X['data'][X['data'].index.isin([100,177])], X['target'][X['target'].index.isin([100,177])])

RandomForestClassifier(n_estimators=10, random_state=42)

In [222]:
y_pred = rf.predict(X['data'])
print(classification_report(X['target'], y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.45      0.97      0.62        71
           2       0.92      0.48      0.63        48

    accuracy                           0.52       178
   macro avg       0.46      0.48      0.42       178
weighted avg       0.43      0.52      0.42       178



d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [223]:
confusion_matrix(X['target'], y_pred) 

array([[ 0, 59,  0],
       [ 0, 69,  2],
       [ 0, 25, 23]], dtype=int64)

In [224]:
lr = LogisticRegression(random_state=42)
lr.fit(X['data'][X['data'].index.isin([0,177])], X['target'][X['target'].index.isin([0,177])])

LogisticRegression(random_state=42)

In [225]:
y_pred_lr = lr.predict(X['data'])
print(classification_report(X['target'], y_pred_lr))

              precision    recall  f1-score   support

           0       0.85      0.90      0.88        59
           1       0.00      0.00      0.00        71
           2       0.37      0.90      0.52        48

    accuracy                           0.54       178
   macro avg       0.41      0.60      0.47       178
weighted avg       0.38      0.54      0.43       178



d:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [226]:
confusion_matrix(X['target'], y_pred_lr) 

array([[53,  0,  6],
       [ 4,  0, 67],
       [ 5,  0, 43]], dtype=int64)

In [278]:
import pickle

pickle.dump(rf, open('saved_models/rf_model_01.pkl', 'wb'))

pickle.dump(lr, open('saved_models/lr_model_01.pkl', 'wb'))

pickle.dump(list(X['data'].columns), open('saved_models/columns_01.pkl', 'wb'))


In [ ]:
import json
## para testear en postman
final = json.dumps([X['data'].loc[90].to_dict(), X['data'].loc[0].to_dict()], indent=2)
print(final)

In [228]:
import requests

In [298]:
response = requests.post("http://127.0.0.1:12345/predict/v1/lr_01", json=[X['data'].loc[0].to_dict(), X['data'].loc[177].to_dict()])
print(response)
print('prediccion: ', response.json())

<Response [200]>
prediccion:  {'model': 'lr_01', 'model_': 'lr_01', 'model__': 'lr_01', 'prediction': '[0, 2]'}


In [297]:
response = requests.post("http://127.0.0.1:12345/predict/v1/rf_01", json=[X['data'].loc[0].to_dict(), X['data'].loc[177].to_dict()])
print(response)
print('prediccion: ', response.json())

<Response [200]>
prediccion:  {'model': 'rf_01', 'model_': 'rf_01', 'model__': 'rf_01', 'prediction': '[1, 2]'}


el monitoreo se abre desde http://localhost:12345/dashboard/login (ojo con el puerto), pass y user admin admin
Monitoreo directo sobre el endpoint piola en http://localhost:12345/dashboard/endpoint/1/hourly_load (endpoint 1 acá)


In [300]:
import random
import time

for i in range(200):
    time.sleep(random.randint(1,3))
    model_ = random.choice(['rf_01', 'lr_01'])
    response = requests.post(f"http://127.0.0.1:12345/predict/v1/{model_}", json=[X['data'].sample(random.randint(0, len(X['data']))).to_dict()])

In [315]:

import sqlite3

# para leer la base de datos que generamos (ver esquema acá https://flask-monitoringdashboard.readthedocs.io/en/latest/developing.html#architecture)


con = sqlite3.connect("flask_monitoringdashboard.db")

requests_df = pd.read_sql_query("SELECT * FROM request", con)

con.close()

requests_df


,id,endpoint_id,duration,time_requested,version_requested,group_by,ip,status_code
0,1,1,9.184361,2022-10-30 23:26:39.085794,1.0,None,127.0.0.1,200
1,2,1,5.230665,2022-10-30 23:26:40.175787,1.0,None,127.0.0.1,200
2,3,1,9.117126,2022-10-30 23:26:40.726398,1.0,None,127.0.0.1,200
3,4,1,11.031866,2022-10-30 23:26:40.917601,1.0,None,127.0.0.1,200
4,5,1,12.999296,2022-10-30 23:26:41.111080,1.0,None,127.0.0.1,200
...,...,...,...,...,...,...,...,...
490,491,1,3.006935,2022-11-01 00:19:38.103014,1.0,None,127.0.0.1,200
491,492,1,3.041267,2022-11-01 00:19:40.122862,1.0,None,127.0.0.1,200
492,493,1,3.041983,2022-11-01 00:19:41.140197,1.0,None,127.0.0.1,200
493,494,1,3.023863,2022-11-01 00:19:43.161365,1.0,None,127.0.0.1,200
